# 06 Stability mismatch proxy
Hypothesis: `E_proxy = feature_drift + (1 - source_retention_acc)` tracks unstable transfer.

## Step 1: Imports and setup
Run safe and naive methods on the same task split.

In [ ]:
from pathlib import Path
import sys
import torch
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
while ROOT != ROOT.parent and not (ROOT / 'src').is_dir():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT / 'src'))

from utils.seed import set_seed
from data.cifar10_transfer import get_cifar10_transfer
from models.transfer_resnet import TransferResNet18
from methods.transfer_learning import pretrain_source, build_transferred_model, run_target_adaptation

FIGS = ROOT / 'outputs' / 'figures'
FIGS.mkdir(parents=True, exist_ok=True)

## Step 2: Build data, pretrain source, and run two strategies
Use gradual unfreeze and naive fine-tune for a controlled comparison.

In [ ]:
SEED = 1
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

set_seed(SEED)
loaders = get_cifar10_transfer(
    data_dir='./data',
    source_classes=[2, 3, 4, 5, 6, 7],
    target_classes=[2, 4, 6],
    source_train_per_class=1000,
    source_test_per_class=300,
    target_train_per_class=40,
    target_test_per_class=300,
    probe_per_class=120,
    batch_size=128,
    num_workers=2,
    seed=SEED,
)

source_model = TransferResNet18(num_classes=loaders.source_num_classes)
pretrain_source(
    model=source_model,
    train_loader=loaders.source_train,
    test_loader=loaders.source_test,
    device=DEVICE,
    epochs=6,
    lr=0.03,
    weight_decay=5e-4,
    momentum=0.9,
    use_progress=True,
);

frames = {}
for method in ['gradual_unfreeze', 'naive_finetune']:
    model, source_head = build_transferred_model(source_model, loaders.target_num_classes)
    result = run_target_adaptation(
        model=model,
        target_train=loaders.target_train,
        target_test=loaders.target_test,
        target_probe=loaders.target_probe,
        source_test=loaders.source_test,
        source_head=source_head,
        device=DEVICE,
        strategy=method,
        epochs=14,
        lr=0.02,
        weight_decay=5e-4,
        momentum=0.9,
        gradual_schedule={
            4: ['backbone.layer4'],
            8: ['backbone.layer3', 'backbone.layer2'],
            11: ['backbone.layer1', 'backbone.bn1', 'backbone.conv1'],
        },
        use_progress=True,
    )
    frames[method] = pd.DataFrame(result.history)

safe = frames['gradual_unfreeze'].copy()
naive = frames['naive_finetune'].copy()
safe['E_proxy'] = safe['feature_drift'] + (1.0 - safe['source_retention_acc'])
naive['E_proxy'] = naive['feature_drift'] + (1.0 - naive['source_retention_acc'])

## Step 3: Plot mismatch proxy and target accuracy
Higher `E_proxy` should align with less reliable adaptation.

In [ ]:
fig, ax = plt.subplots(figsize=(6.5, 3.6))
ax.plot(safe['epoch'], safe['E_proxy'], marker='o', label='gradual_unfreeze')
ax.plot(naive['epoch'], naive['E_proxy'], marker='o', label='naive_finetune')
ax.set_title('Mismatch proxy over adaptation')
ax.set_xlabel('epoch')
ax.set_ylabel('E_proxy')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
fig.savefig(FIGS / '06_e_proxy_by_method.png', dpi=150, bbox_inches='tight')

fig, ax = plt.subplots(figsize=(6.5, 3.6))
ax.plot(safe['epoch'], safe['target_test_acc'], marker='o', label='gradual_unfreeze')
ax.plot(naive['epoch'], naive['target_test_acc'], marker='o', label='naive_finetune')
ax.set_title('Target accuracy under matched setup')
ax.set_xlabel('epoch')
ax.set_ylabel('target_test_acc')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
fig.savefig(FIGS / '06_target_acc_safe_vs_naive.png', dpi=150, bbox_inches='tight')

In [ ]:
stability = pd.DataFrame({
    'epoch': safe['epoch'],
    'E_safe': safe['E_proxy'],
    'E_naive': naive['E_proxy'],
    'target_safe': safe['target_test_acc'],
    'target_naive': naive['target_test_acc'],
})
stability.tail()

### Expected Outcome
Naive fine-tuning should maintain a higher `E_proxy` than gradual unfreezing.

## Usage
If `E_proxy` rises while target gains stall, reduce adaptation aggressiveness.